# Evacutation Model

* Purpose: What impacts the speed of evacuating a room?
* Humans move on a grid and seek for exits
* Humans may help each other
* Simpler version of the original model

In [ ]:
import sys
sys.path.insert(0,'../../abmodel')

from mesa.visualization import SolaraViz, make_space_component, make_plot_component
from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human, FireExit, Wall, Sight
import os
import solara

In [ ]:
current_dir = '../../abmodel'

# Specify the parameters changeable by the user, in the web interface
model_params = {
    "random_spawn": {
        "type": "Checkbox",
        "value": True,
        "label": "Random spawn of initial positions",
    },
    "floor_size": {
        "type": "SliderInt",
        "value": 12,
        "label": "Room size (edge)",
        "min": 5,
        "max": 30,
        "step": 1,
    },
    "human_count": {
        "type": "SliderInt",
        "value": 80,
        "label": "Number Of Human Agents",
        "min": 1,
        "max": 500,
        "step": 5,
    },
    "max_speed": {
        "type": "SliderInt",
        "value": 2,
        "label": "Maximum Speed of agents",
        "min": 1,
        "max": 5,
        "step": 1,
    },
    "alarm_believers_prop": {
        "type": "SliderFloat",
        "value": 1.0,
        "label": "Proportion of Alarm Believers",
        "min": 0.0,
        "max": 1.0,
        "step": 0.05,
    },
    "cooperation_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Cooperation",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
    "nervousness_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Nervousness",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
}

def agent_portrayal(agent):
    size = 10
    if type(agent) is Human:
        if agent.believes_alarm:
            # believes in alarm
            shape = os.path.join(current_dir, "fire_evacuation/resources/alarmbeliever.png")
        elif agent.nervousness > Human.NERVOUSNESS_PANIC_THRESHOLD:
            shape = os.path.join(current_dir, "fire_evacuation/resources/panicked_human.png")
        elif agent.humantohelp is not None:
            shape = os.path.join(current_dir, "fire_evacuation/resources/cooperating_human.png")
        else:
            shape = os.path.join(current_dir, "fire_evacuation/resources/human.png")
    elif type(agent) is FireExit:
        shape = os.path.join(current_dir, "fire_evacuation/resources/fire_exit.png")
    elif type(agent) is Wall:
        shape = os.path.join(current_dir, "fire_evacuation/resources/wall.png")
    elif type(agent) is Sight:
        shape = os.path.join(current_dir, "fire_evacuation/resources/eye.png")
    else:
        shape = "X"
    return {"size": size,
            "marker": shape,
            "color": "red",
            }

model = solara.reactive(FireEvacuation(
            floor_size = 14,
            human_count = 70,
            alarm_believers_prop = 1.0,
            max_speed = 2,
            seed = 3)
        )

In [ ]:
page = SolaraViz(
    model,
    model_params = model_params,
    name="Evacuation Model",
    components=[make_space_component(agent_portrayal),
                make_plot_component("AvgNervousness"),
                ],
)

page  # noqa

## FireEvacuation(Model)

In [ ]:
# not for running
MIN_SPEED = 0
MAX_SPEED = 3

EXTRA_STEPS_PER_IMMOBILE = 10

COOPERATE_WO_EXIT = False
AlARM_BELIEVERS_PROB = 0.9
TURN_WHEN_BLOCKED_PROB = 0.5
COOPERATION_MEAN = 0.3
NERVOUSNESS_MEAN = 0.3

In [ ]:
# not for running
def __init__(
    self,
    floor_size: int = 14,
    human_count: int = 50,
    visualise_vision = True,
    random_spawn = True,
    alarm_believers_prop = AlARM_BELIEVERS_PROB,
    turnwhenblocked_prop = TURN_WHEN_BLOCKED_PROB,
    max_speed = 1,
    cooperation_mean = COOPERATION_MEAN,
    nervousness_mean = NERVOUSNESS_MEAN,
    seed = 1,
 ):

    super().__init__(seed=seed)
    self.rng = np.random.default_rng(seed)
    
    if human_count > floor_size ** 2:
        raise ValueError("Number of humans to high for the room!")

    self.MAX_SPEED = max_speed
    self.COOPERATE_WO_EXIT = FireEvacuation.COOPERATE_WO_EXIT
    
    self.stepcounter = -1
    
    # Create floorplan
    floorplan = np.full((floor_size, floor_size), '_')
    floorplan[(0,-1),:]='W'
    floorplan[:,(0,-1)]='W'
    floorplan[math.floor(floor_size/2),(0,-1)] = 'E'
    floorplan[(0,-1), math.floor(floor_size/2)] = 'E'

    # Rotate the floorplan so it's interpreted as seen in the text file
    floorplan = np.rot90(floorplan, 3)

    # Init params
    self.width = floor_size
    self.height = floor_size
    self.human_count = human_count
    self.visualise_vision = visualise_vision

    # Set up grid
    self.grid = MultiGrid(floor_size, floor_size, torus=False)

    # Used to easily see if a location is a FireExit or Door, since this needs to be done a lot
    self.fire_exits: dict[Coordinate, FireExit] = {}
    self.doors: dict[Coordinate, Door] = {}

    # If random spawn is false, spawn_pos_list will contain the list of possible 
    # spawn points according to the floorplan
    self.random_spawn = random_spawn
    self.spawn_pos_list: list[Coordinate] = []

    self.decisioncount = dict()
    
    # Load floorplan objects
    for (x, y), value in np.ndenumerate(floorplan):
        pos: Coordinate = (x, y)

        value = str(value)
        floor_object = None
        if value == "W":
            floor_object = Wall(self)
        elif value == "E":
            floor_object = FireExit(self)
            self.fire_exits[pos] = floor_object
        elif value == "S":
            self.spawn_pos_list.append(pos)
        if floor_object:
            self.grid.place_agent(floor_object, pos)

    # Create a graph of traversable routes, used by humans for pathing
    self.graph = nx.Graph()
    for agents, pos in self.grid.coord_iter():
        # If the location is empty, or there are no non-traversable objects
        if len(agents) == 0 or not any(not agent.traversable for agent in agents):
            neighbors_pos = self.grid.get_neighborhood(
                pos, moore=True, include_center=True, radius=1
            )

            for neighbor_pos in neighbors_pos:
                # If the neighbour position is empty, or no non-traversable 
                # contents, add an edge
                if self.grid.is_cell_empty(neighbor_pos) or not any(
                    not agent.traversable
                    for agent in self.grid.get_cell_list_contents(neighbor_pos)
                ):
                    self.graph.add_edge(pos, neighbor_pos)

    # Collects statistics from our model run
    self.datacollector = DataCollector(
        {
            "NumEscaped" : lambda m: self.get_num_escaped(m),
            "AvgNervousness": lambda m: self.get_human_nervousness(m),
            "AvgSpeed": lambda m: self.get_human_speed(m),
         }
    )
    
    # Start placing human humans
    for _i in range(0, self.human_count):
        if self.random_spawn:  # Place human humans randomly
            pos = tuple(self.rng.choice(tuple(self.grid.empties)))
        else:  # Place human humans at specified spawn locations
            pos = self.rng.choice(self.spawn_pos_list)

        if pos:
            # Create a random human
            speed = self.rng.integers(self.MIN_SPEED, self.MAX_SPEED + 1)

            nervousness = -1
            while nervousness < 0 or nervousness > 1:
                nervousness = self.rng.normal(loc = nervousness_mean, scale = 0.2)
                
            cooperativeness = -1
            while cooperativeness < 0 or cooperativeness > 1:
                cooperativeness = self.rng.normal(cooperation_mean)

            belief_distribution = [alarm_believers_prop, 1 - alarm_believers_prop]
            believes_alarm = self.rng.choice([True, False], p=belief_distribution)

            orientation = Human.Orientation(self.rng.integers(1,5))
            
            # decide here whether to add a facilitator
            
            human = Human(
                speed=speed,
                orientation=orientation,
                nervousness=nervousness,
                cooperativeness=cooperativeness,
                believes_alarm=believes_alarm,
                turnwhenblocked_prop = turnwhenblocked_prop,
                model=self,
            )

            self.grid.place_agent(human, pos)
        else:
            print("No tile empty for human placement!")

    self.running = True


In [ ]:
# not for running
def step(self):
    """
    Advance the model by one step.
    """

    self.agents.shuffle_do("step")
    self.datacollector.collect(self)

    # If all humans escaped, stop the model and collect the results
    if self.get_num_escaped(self) == self.human_count:
        self.running = False
    
    # In case there are Humans unable to move let the model run extra steps
    # to indicate the insufficient escape: 
    if self.stepcounter == 0:
        self.running = False
    elif self.stepcounter > 0:
        self.stepcounter -=1
    elif self.get_human_speed(self) == 0:
        self.stepcounter = FireEvacuation.EXTRA_STEPS_PER_IMMOBILE *\
            sum(map(lambda agent : isinstance(agent, Human) 
                        and not agent.escaped, self.agents))
            
def run(self, n):
    """Run the model for n steps."""
    for _ in range(n):
        self.step()

## Human(Agent)

In [ ]:
# not for running
def step(self):
    if not self.escaped and self.pos:
        self.turned = False

        ######################
        # Update properties
        ######################

        # update nervousness:
        self.update_nervousness()

        # update field of vision
        self.learn_fieldofvision()

        # update speed
        if self.nervousness > Human.NERVOUSNESS_SPEEDCHANGE_THRESHOLD:
            # Either slow down or accelerate in panic situation:
            self.speed = int(min(max(Human.MIN_SPEED, self.speed + self.model.rng.choice([-1, 1])), Human.MAX_SPEED)) 

        ######################
        # Decide action:
        ######################

        # check panic mode
        if self.nervousness > Human.NERVOUSNESS_PANIC_THRESHOLD:
            if self.model.rng.random() < Human.RANDOMWALK_PROB:
                # print(str(self.pos) + "Random target because of panic: " + str(self.planned_target[1]))
                self.get_random_target()

        else:        
            # check cooperation
            if self.cooperativeness > self.COOPERATIVENESS_THRESHOLD and self.humantohelp == None \
                    and (len(self.exits) > 0 
                    or self.model.COOPERATE_WO_EXIT):
                self.cooperate()

            # If the agent believes the alarm, attempt to plan 
            # an exit location if we haven't already and we aren't performing an action
            if not self.turned and not isinstance(self.planned_target, FireExit) and not isinstance(self.planned_target, Human):
                if self.believes_alarm:
                    self.attempt_exit_plan()
                    #print("Human (" + str(self.pos[0]) + "/" + str(self.pos[1])+ "): Planned target: " + self.get_planned_target())


        ######################
        # Perform action:
        ######################

        if not self.turned:
            if self.planned_target == None:
                self.get_random_target()


            # finally go
            self.move_toward_target()

            self.help()

            # Agent reached a fire escape, proceed to exit
            if self.pos in self.model.fire_exits.keys():
                self.escaped = True
                self.model.grid.remove_agent(self)

In [ ]:
# not for running
def help(self):
    if self.humantohelp != None:
        if self.humantohelp.escaped:
            self.humantohelp = None
            self.planned_target = None
        # reached human to help?
        elif self.humantohelp in self.model.grid.get_neighbors(self.pos, moore=True):
            self.humantohelp.nervousness -= Human.NERVOUSNESS_DECREASE_HELP
            self.humantohelp.nervousness = min(max(0.0, self.humantohelp.nervousness), 1.0) 
            if self.humantohelp.speed == 0:
                self.humantohelp.speed = 1
            elif not self.humantohelp.believes_alarm:
                self.humantohelp.believes_alarm = True
            elif len(self.exits) > 0:
                self.humantohelp.exits = self.exits
            self.humantohelp = None
            self.planned_target = None

In [ ]:
# not for running
def attempt_exit_plan(self):
    self.planned_target = None

    if len(self.exits) > 0:
        if len(self.exits) > 1:  
            # If there is more than one exit known
            best_distance = None
            for exitdoor in self.exits.keys():
                # Let's use Bresenham's to find the 'closest' exit
                length = len(get_line(self.pos, exitdoor.pos))
                if not best_distance or length < best_distance:
                    best_distance = length
                    self.planned_target = exitdoor

        else:
            self.planned_target = list(self.exits.keys())[0]

    elif self.turned == False:
        # If there's no fire-escape in sight, turn around
        self.turn()  